# Testing plotting swap curves with bqplot
*Author: Alexander Nadjalin*

*Date:   2020-01-31*

Simple example of interactive plotting using Bloombergs bqplot module for swap curves at two different dates.

### Modules and file imports

In [156]:
import pandas as pd
from pathlib import Path
from bqplot import *
from IPython.display import display, clear_output
import ipywidgets as widgets

def read_time_series(name: str) -> pd.DataFrame:
    """

    Read .csv file by name from directory "/eikon_time_series_files".
    :param name: File name (including suffix).
    :return: Pandas DataFrame.
    """
    import_dir = Path.cwd().joinpath('eikon_time_series_files')

    path = Path.joinpath(import_dir, Path(name))
    if path.exists():
        return pd.read_csv(path, sep=',', index_col=0, parse_dates=False)
    else:
        print('File type "' + name + '.csv' + ' does not exist. Aborted.')
        quit()

### File definition and import

In [157]:
# File name to save time series as (including ".csv" ending)
name_time_series = 'Swaps_time_series.csv'

# Read time series:
df_ts = read_time_series(name_time_series)

### Plot of swap curves for two dates including differences

In [158]:
def plot(end_date: str, start_date: str) -> None:
    """
    
    Plot two lines, one for each selected dates.
    Plot a bar plot representing the difference between end_date values and start_date values.
    :param end_date: Later date.
    :param start_date: Earlier date.
    :return: None.
    """
    x_data = df_ts.columns.values.tolist()

    df_diff = df_ts.loc[end_date, :] - df_ts.loc[start_date, :]
    y_end = df_ts.loc[end_date, :].values.flatten()
    y_start = df_ts.loc[start_date, :].values.flatten()
    
    x_sc = OrdinalScale()
    y_sc = LinearScale()
    ax_x = Axis(label='RIC', scale=x_sc, grid_lines='solid', tick_rotate=90, tick_style={'font-size': 7}, label_offset='40',
               offset={'scale':x_sc, 'value':10})
    ax_y = Axis(label='%', scale=y_sc, orientation='vertical',tick_format='0.2f', tick_style={'font-size': 9}, label_offset='-40')

    line = Lines(x=x_data, y=[y_end, y_start], scales={'x': x_sc, 'y': y_sc}, 
                 display_legend=True, labels=[end_date, start_date], stroke_width=1)
    bar = Bars(x=x_data, y=df_diff.tolist(), scales={'x': x_sc, 'y': y_sc}, padding=0.5)

    fig = Figure(marks=[line, bar], axes=[ax_x, ax_y], title='Swap rates and differences', legend_location='top-left')
    display(fig)

### Widget definition and handlers

In [160]:
# Initial values
dates = df_ts.index.values
start_date = dates[0]
end_date = dates[-1]

# Widget start date
end_widget = widgets.Dropdown(
    options=dates,
    value = end_date,
    description='End date:')
display(end_widget)

# Widget for end date
start_widget = widgets.Dropdown(
    options=dates,
    value = start_date,
    description='Start date:')
display(start_widget)

# Plot with initial values
plot(end_date, start_date)

# Change handler for end_date
def on_change_end(change):
    if change['name'] == 'value' and (change['new'] != change['old']):
        end_date = change['new']
        # Refresh widgets and resulting plot
        clear_output()
        display(end_widget)
        display(start_widget)
        plot(end_date, start_date)

# Change handler for end_date
def on_change_start(change):
    if change['name'] == 'value' and (change['new'] != change['old']):
        start_date = change['new']
        # Refresh widgets and resulting plot
        clear_output()
        display(end_widget)
        display(start_widget)
        plot(end_date, start_date)

# Capture changed values in dropdowns
end_widget.observe(on_change_end)
start_widget.observe(on_change_start)

Dropdown(description='End date:', index=271, options=('2019-01-10', '2019-01-11', '2019-01-14', '2019-01-15', …

Dropdown(description='Start date:', options=('2019-01-10', '2019-01-11', '2019-01-14', '2019-01-15', '2019-01-…

Figure(axes=[Axis(label='RIC', label_offset='40', offset={'scale': OrdinalScale(), 'value': 10}, scale=Ordinal…